In [1]:

import sqlalchemy as sqla
from sqlalchemy import text
import os
from dotenv import load_dotenv
import pandas as pd

In [2]:
# connecting to datatbase

load_dotenv() # loading env variables

# Access environment variables
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME2 = os.getenv("DB_NAME2")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")

engine = sqla.create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME2}")

In [3]:
# TC1: How many rows are in the bookings table | EASY
with engine.begin() as conn:
    query = text("""select count(*) from bookings.bookings""")
    df = pd.read_sql_query(query, conn)
df

,count
0,593433


In [4]:
# TC2: How many rows are in the aircrafts table | EASY
with engine.begin() as conn:
    query = text("""select count(*) from bookings.aircrafts_data""")
    df = pd.read_sql_query(query, conn)
df

,count
0,9


In [5]:
# TC3: find the top ten most expensive bookings and order total amounts in descending order | EASY
with engine.begin() as conn:
    query = text("""SELECT   *
                    FROM bookings
                    ORDER BY total_amount desc
                    LIMIT 10;""")
    df = pd.read_sql_query(query, conn)
df

,book_ref,book_date,total_amount
0,3B54BB,2017-07-05 14:08:00+00:00,1204500.0
1,53B75C,2017-05-29 06:20:00+00:00,1116700.0
2,3AC131,2017-07-30 22:06:00+00:00,1087100.0
3,D7061C,2017-06-06 08:49:00+00:00,1065600.0
4,65A6EA,2017-07-03 03:28:00+00:00,1065600.0
5,4B58DB,2017-05-14 12:39:00+00:00,1065600.0
6,D7E9AA,2017-08-08 02:29:00+00:00,1062800.0
7,EF479E,2017-08-02 12:58:00+00:00,1035100.0
8,03BCC9,2017-06-26 03:26:00+00:00,1022500.0
9,1BE923,2017-06-24 10:29:00+00:00,1016500.0


In [6]:
# TC4: How many ticket bookings does passenger Antonina Kuznecova have?  | EASY
with engine.begin() as conn:
    query = text("""select count(*) from bookings.tickets where passenger_name in ('ANTONINA KUZNECOVA')""")
    df = pd.read_sql_query(query, conn)
df

,count
0,117


In [7]:
# TC5: How many ticket bookings does passenger Antonina Kuznecova have (testing for case sensitivity)?  | EASY
with engine.begin() as conn:
    query = text("""select count(*) from bookings.tickets where passenger_name in ('Antonina Kuznecova')""")
    df = pd.read_sql_query(query, conn)
df

,count
0,0


In [7]:
# TC6: How many distinct flights departed from Moscow? (testing to determine if langchain can Identify db views)  | EASY
with engine.begin() as conn:
    query = text("""select count(distinct flight_no) from routes where departure_city in ('Moscow')""")
    df = pd.read_sql_query(query, conn)
df

,count
0,154


In [8]:
# TC7: Get me the count of each flight status grouped by status and also provide the minimum and maximum departure ordered by the minimum scheduled
#      departure | MEDIUM
with engine.begin() as conn:
    query = text("""SELECT   status,
                            count(*) as count,
                            min(scheduled_departure) as min_scheduled_departure,
                            max(scheduled_departure) as max_scheduled_departure
                    FROM     flights
                    GROUP BY status 
                    ORDER BY min_scheduled_departure;""")
    df = pd.read_sql_query(query, conn)
df

,status,count,min_scheduled_departure,max_scheduled_departure
0,Arrived,49235,2017-05-16 23:00:00+00:00,2017-08-15 14:25:00+00:00
1,Cancelled,429,2017-05-17 16:10:00+00:00,2017-09-14 17:55:00+00:00
2,Departed,58,2017-08-15 06:55:00+00:00,2017-08-15 14:50:00+00:00
3,Delayed,41,2017-08-15 12:15:00+00:00,2017-08-16 14:25:00+00:00
4,On Time,518,2017-08-15 14:55:00+00:00,2017-08-16 15:00:00+00:00
5,Scheduled,15383,2017-08-16 15:05:00+00:00,2017-09-14 17:40:00+00:00


In [10]:
# TC8: Get me the count of each flight status grouped by status. Return only the status and count for each status type | MEDIUM
with engine.begin() as conn:
    query = text("""SELECT   status,
                            count(*) as count
                    FROM     flights
                    GROUP BY status ;""")
    df = pd.read_sql_query(query, conn)
df

,status,count
0,Departed,58
1,Arrived,49235
2,On Time,518
3,Cancelled,429
4,Delayed,41
5,Scheduled,15383


In [11]:
# TC9: Get me the list of seats and the fare conditions or class where those seats are for aircraft model Cessna 208 Caravan and output the aircraft code
# the model, the seat number, and the fare conditions and order it by the seat number  | MEDIUM
with engine.begin() as conn:
    query = text("""SELECT   a.aircraft_code,
                    a.model ->> 'en' AS model,
                    s.seat_no,
                    s.fare_conditions
                    FROM aircrafts_data a
                    JOIN seats s ON a.aircraft_code = s.aircraft_code
                    WHERE a.model ->> 'en'  like 'Cessna 208 Caravan'
                    ORDER BY s.seat_no;""")
    df = pd.read_sql_query(query, conn)
df

,aircraft_code,model,seat_no,fare_conditions
0,CN1,Cessna 208 Caravan,1A,Economy
1,CN1,Cessna 208 Caravan,1B,Economy
2,CN1,Cessna 208 Caravan,2A,Economy
3,CN1,Cessna 208 Caravan,2B,Economy
4,CN1,Cessna 208 Caravan,3A,Economy
5,CN1,Cessna 208 Caravan,3B,Economy
6,CN1,Cessna 208 Caravan,4A,Economy
7,CN1,Cessna 208 Caravan,4B,Economy
8,CN1,Cessna 208 Caravan,5A,Economy
9,CN1,Cessna 208 Caravan,5B,Economy


In [3]:
# TC10: Get me the passenger name that had the most flights that had a status of Arrived or Departed. The output should return the passenger name and 
# a count of how many distinct flights this passenger had   | MEDIUM
with engine.begin() as conn:
    query = text("""select  a.passenger_name , count(distinct c.flight_no) as total_flights
                    from 	tickets a 
                            join ticket_flights b
                            on a.ticket_no = b.ticket_no
                            join flights c
                            on b.flight_id = c.flight_id
                    where status = 'Arrived' or status = 'Departed'
                    group by a.passenger_name
                    order by total_flights desc
                    limit 1;""")
    df = pd.read_sql_query(query, conn)
df

,passenger_name,total_flights
0,ALEKSANDR KUZNECOV,407


In [13]:
# TC11: Get me the the passenger name, phone number, and email of 10 passengers ordered by passenger name in ascending order. 
# Make sure to only retrieve data of passengers that have an email  | MEDIUM
with engine.begin() as conn:
    query = text("""select a.ticket_no ,a.passenger_name , a.contact_data ->> 'phone' as phone_number 
                        ,a.contact_data ->> 'email' as email
                    from tickets a 
                    where a.contact_data ->> 'email' is not null
                    order by passenger_name
                    limit 10;""")
    df = pd.read_sql_query(query, conn)
df

,ticket_no,passenger_name,phone_number,email
0,0005433102870,ADELINA AFANASEVA,+70868634381,afanaseva_adelina-1960@postgrespro.ru
1,0005434029641,ADELINA ALEKSEEVA,+70540852393,adelina_alekseeva-021969@postgrespro.ru
2,0005433399027,ADELINA ANDREEVA,+70068029304,andreeva-a_1958@postgrespro.ru
3,0005432186838,ADELINA BELYAEVA,+70236254085,adelina.belyaeva.1956@postgrespro.ru
4,0005433349099,ADELINA CHERNOVA,+70537037190,adelina_chernova041972@postgrespro.ru
5,0005433283191,ADELINA DANILOVA,+70595112786,danilova_adelina-18121972@postgrespro.ru
6,0005433609492,ADELINA DENISOVA,+70243788454,denisova.a-01081973@postgrespro.ru
7,0005435419725,ADELINA EFIMOVA,+70920616014,a.efimova.31051981@postgrespro.ru
8,0005435096669,ADELINA EGOROVA,+70996111276,adelina_egorova15081969@postgrespro.ru
9,0005433679505,ADELINA FILIPPOVA,+70571835825,filippova.adelina_041975@postgrespro.ru


In [8]:
# TC12: Which cities have more than 1 airport? Return the airport code, airport name, and city in your response.  | MEDIUM
with engine.begin() as conn:
    query = text("""SELECT  a.airport_code as code,
                            a.airport_name ->> 'en' as airport_name,
                            a.city ->> 'en' as city
                    FROM    airports_data a
                    WHERE   a.city ->> 'en' IN (
                                SELECT   aa.city ->> 'en' as city
                                FROM     airports_data aa
                                GROUP BY aa.city
                                HAVING   COUNT(*) > 1
                            )
                    ORDER BY a.city, a.airport_code;""")
    df = pd.read_sql_query(query, conn)
df

,code,airport_name,city
0,DME,Domodedovo International Airport,Moscow
1,SVO,Sheremetyevo International Airport,Moscow
2,VKO,Vnukovo International Airport,Moscow
3,ULV,Ulyanovsk Baratayevka Airport,Ulyanovsk
4,ULY,Ulyanovsk East Airport,Ulyanovsk


In [17]:
# TC13: Get me the ticket number, flight id ,fare conditions, seat number, boarding number, passenger name, and amount for flight 4686 where
# the status is Arrived. Order the results by amount in descending order    | HARD
with engine.begin() as conn:
    query = text("""select distinct tf.ticket_no, tf.flight_id, tf.fare_conditions, 
                                    bp.seat_no, bp.boarding_no, t.passenger_name, tf.amount
                    from ticket_flights tf
                                    left join boarding_passes bp 
                                    on tf.ticket_no = bp.ticket_no
                                    and tf.flight_id = bp.flight_id
                                    left join tickets t
                                    on tf.ticket_no = tf.ticket_no
                                    and bp.ticket_no = t.ticket_no
                                    left join flights f
                                    on tf.flight_id = f.flight_id
                    where tf.flight_id = 4686 and f.status = 'Arrived'
                    order by tf.amount desc;""")
    df = pd.read_sql_query(query, conn)
df


,ticket_no,flight_id,fare_conditions,seat_no,boarding_no,passenger_name,amount
0,0005432655571,4686,Business,2C,29,ALEKSEY MOISEEV,184500.0
1,0005432869497,4686,Business,4A,130,VALENTINA ORLOVA,184500.0
2,0005432869500,4686,Business,5B,74,SERGEY NIKOLAEV,184500.0
3,0005432984785,4686,Business,2F,197,ANTONINA GRISHINA,184500.0
4,0005432984890,4686,Business,4B,131,KLAVDIYA STEPANOVA,184500.0
...,...,...,...,...,...,...,...
213,0005435982479,4686,Economy,22E,51,IRINA NIKOLAEVA,61500.0
214,0005435982480,4686,Economy,22H,178,NATALYA KISELEVA,61500.0
215,0005435982481,4686,Economy,35D,162,OLGA MASLOVA,61500.0
216,0005435982483,4686,Economy,38D,28,ALEKSANDR PAVLOV,61500.0


In [9]:
# TC14: Which flight segments are included into ticket number 0005432661915? Please include the scheduled departure date only, the departure city along with the departure airport
# the arrival city along with the arrival airport, the status of the flight, and the seat numbers for each flight segment. Please output order by scheduled
# departure | HARD
with engine.begin() as conn:
    query = text("""SELECT  to_char(f.scheduled_departure, 'DD.MM.YYYY') AS when,
                            ad.city ->> 'en' || ' (' || f.departure_airport || ')' AS departure,
                            ad2.city ->> 'en' || ' (' || f.arrival_airport || ')' AS arrival,
                            f.status,
                            bp.seat_no
                    FROM    ticket_flights tf
                            JOIN flights f 
                            ON tf.flight_id = f.flight_id
                            JOIN airports_data ad
                            ON f.departure_airport = ad.airport_code
                            JOIN airports_data ad2
                            ON f.arrival_airport = ad2.airport_code
                            JOIN boarding_passes bp
                            ON tf.flight_id = bp.flight_id 
                            AND tf.ticket_no = bp.ticket_no
                    WHERE tf.ticket_no = '0005432661915'
                    ORDER BY f.scheduled_departure;""")
    df = pd.read_sql_query(query, conn)
df

,when,departure,arrival,status,seat_no
0,29.07.2017,Moscow (SVO),Anadyr (DYR),Arrived,5C
1,01.08.2017,Anadyr (DYR),Khabarovsk (KHV),Arrived,1D
2,02.08.2017,Khabarovsk (KHV),Blagoveschensk (BQS),Arrived,2C
3,08.08.2017,Blagoveschensk (BQS),Khabarovsk (KHV),Arrived,2D
4,11.08.2017,Khabarovsk (KHV),Anadyr (DYR),Arrived,20B


In [19]:
# TC15: which flight segments are included into ticket number 0005432661915? Please include the scheduled departure date only, the departure city along with the departure airport
# the arrival city along with the arrival airport, the fare conditions or class, and the amount each transaction cost. Please order output by scheduled
# departure | HARD
with engine.begin() as conn:
    query = text("""SELECT to_char(f.scheduled_departure, 'DD.MM.YYYY') AS when,
                    ad.city ->> 'en' || ' (' || f.departure_airport || ')' AS departure,
                    ad2.city ->> 'en' || ' (' || f.arrival_airport || ')' AS arrival,
                    tf.fare_conditions AS class,
                    tf.amount
                    FROM     ticket_flights tf
                    JOIN flights f 
                    ON tf.flight_id = f.flight_id
                    JOIN airports_data ad
                    ON f.departure_airport = ad.airport_code
                    JOIN airports_data ad2
                    ON f.arrival_airport = ad2.airport_code
                    WHERE tf.ticket_no = '0005432661915'
                    ORDER BY f.scheduled_departure;""")
    df = pd.read_sql_query(query, conn)
df

,when,departure,arrival,class,amount
0,29.07.2017,Moscow (SVO),Anadyr (DYR),Business,185300.0
1,01.08.2017,Anadyr (DYR),Khabarovsk (KHV),Business,92200.0
2,02.08.2017,Khabarovsk (KHV),Blagoveschensk (BQS),Business,18000.0
3,08.08.2017,Blagoveschensk (BQS),Khabarovsk (KHV),Business,18000.0
4,11.08.2017,Khabarovsk (KHV),Anadyr (DYR),Economy,30700.0
5,16.08.2017,Anadyr (DYR),Moscow (SVO),Business,185300.0


In [20]:
# TC16: Get me the aircraft code and the fare conditions along with each fare conditions count for each aircraft code. For example, aircraft code 319
# would have a fare conditions of business(20), Economy(96) where 20 and 96 represent the number of seats in those conditions.
# The output should have the aircraft_code, and fare_conditions with their seat count in the same record.  | HARD
with engine.begin() as conn:
    query = text("""SELECT s2.aircraft_code,
                    string_agg (s2.fare_conditions || '(' || s2.num::text || ')',
                                ', ') as fare_conditions
                    FROM     (
                            SELECT   s.aircraft_code, s.fare_conditions, count(*) as num
                            FROM     seats s
                            GROUP BY s.aircraft_code, s.fare_conditions
                            ORDER BY s.aircraft_code, s.fare_conditions
                            ) s2
                    GROUP BY s2.aircraft_code
                    ORDER BY s2.aircraft_code;""")
    df = pd.read_sql_query(query, conn)
df

,aircraft_code,fare_conditions
0,319,"Business(20), Economy(96)"
1,320,"Business(20), Economy(120)"
2,321,"Business(28), Economy(142)"
3,733,"Business(12), Economy(118)"
4,763,"Business(30), Economy(192)"
5,773,"Business(30), Comfort(48), Economy(324)"
6,CN1,Economy(12)
7,CR2,Economy(50)
8,SU9,"Business(12), Economy(85)"
